# Import the required libraries

from bs4 import BeautifulSoup
import requests
import urllib2
import pandas as pd

# Scrape data from website

In [140]:
quote_page = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#with open (quote_page) as html_file:
    #soup = BeautifulSoup(html_file,'lxml')
page = urllib.urlopen(quote_page)
soup = BeautifulSoup(page, 'html.parser')
#print(soup.prettify())





# Extract table data

In [87]:
#match = soup.tbody.text
#match = soup.find('table', attrs={'class':'wikitable sortable '})
#class="wikitable sortable jquery-tablesorter"
#match1 = match.find_all('tr')
#print(match)
table = soup.find('tbody')
table_rows = table.find_all('tr')
res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)
df = pd.DataFrame(res, columns=["PCODE", "BOROUGH", "NEIGHBOURHOOD"])
print(df)


    PCODE           BOROUGH                                     NEIGHBOURHOOD
0     M1A      Not assigned                                      Not assigned
1     M2A      Not assigned                                      Not assigned
2     M3A        North York                                         Parkwoods
3     M4A        North York                                  Victoria Village
4     M5A  Downtown Toronto                                      Harbourfront
5     M5A  Downtown Toronto                                       Regent Park
6     M6A        North York                                  Lawrence Heights
7     M6A        North York                                    Lawrence Manor
8     M7A      Queen's Park                                      Not assigned
9     M8A      Not assigned                                      Not assigned
10    M9A         Etobicoke                                  Islington Avenue
11    M1B       Scarborough                                     

# Remove Not Assigned from "BOROUGH column"

In [110]:
df.drop(df.loc[df['BOROUGH']=='Not assigned'].index, inplace=True)
df.reset_index(drop = True, inplace = True)
print(df)

    PCODE           BOROUGH                                     NEIGHBOURHOOD
0     M3A        North York                                         Parkwoods
1     M4A        North York                                  Victoria Village
2     M5A  Downtown Toronto                                      Harbourfront
3     M5A  Downtown Toronto                                       Regent Park
4     M6A        North York                                  Lawrence Heights
5     M6A        North York                                    Lawrence Manor
6     M7A      Queen's Park                                      Not assigned
7     M9A         Etobicoke                                  Islington Avenue
8     M1B       Scarborough                                             Rouge
9     M1B       Scarborough                                           Malvern
10    M3B        North York                                   Don Mills North
11    M4B         East York                                  Woo

# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 6th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [125]:
Index1 = df.loc[df['NEIGHBOURHOOD']=='Not assigned'].index
print (Index1)
df['NEIGHBOURHOOD'].loc[Index1] = df['BOROUGH'].loc[Index1]
print(df)

Int64Index([], dtype='int64')
    PCODE           BOROUGH                                     NEIGHBOURHOOD
0     M3A        North York                                         Parkwoods
1     M4A        North York                                  Victoria Village
2     M5A  Downtown Toronto                                      Harbourfront
3     M5A  Downtown Toronto                                       Regent Park
4     M6A        North York                                  Lawrence Heights
5     M6A        North York                                    Lawrence Manor
6     M7A      Queen's Park                                      Queen's Park
7     M9A         Etobicoke                                  Islington Avenue
8     M1B       Scarborough                                             Rouge
9     M1B       Scarborough                                           Malvern
10    M3B        North York                                   Don Mills North
11    M4B         East York       

# More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [137]:
grouped = df.groupby(['PCODE','BOROUGH'])
rows = []
for name,group in grouped:
    row = [ name[0], name[1], (', '.join(group['NEIGHBOURHOOD']) )]
    rows.append(row)
final_df = pd.DataFrame(rows,columns=["PCODE", "BOROUGH", "NEIGHBOURHOOD"])
final_df

,PCODE,BOROUGH,NEIGHBOURHOOD
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


# In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [135]:
final_df.shape

(103, 3)

#  Import csv from Coursera for coordinates of Postalcodes.

In [154]:
import pandas as pd
import requests

url="https://cocl.us/Geospatial_data"
Coursera_DataFrame=pd.read_csv(url)

# Join coordinates on the Postalcode of with my data

In [155]:
final_df1 = final_df.set_index('PCODE').join(Coursera_DataFrame.set_index('Postal Code')).reset_index(drop=True)
final_df1.head()

,BOROUGH,NEIGHBOURHOOD,Latitude,Longitude
0,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,Scarborough,Woburn,43.770992,-79.216917
4,Scarborough,Cedarbrae,43.773136,-79.239476


# Get toronto data

In [157]:

Toronto = final_df1[ final_df1['BOROUGH'].str.contains('Toronto', case=False)].reset_index(drop=True)  
Toronto

,BOROUGH,NEIGHBOURHOOD,Latitude,Longitude
0,East Toronto,The Beaches,43.676357,-79.293031
1,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,East Toronto,Studio District,43.659526,-79.340923
4,Central Toronto,Lawrence Park,43.728020,-79.388790
5,Central Toronto,Davisville North,43.712751,-79.390197
6,Central Toronto,North Toronto West,43.715383,-79.405678
7,Central Toronto,Davisville,43.704324,-79.388790
8,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
9,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West",43.686412,-79.400049


# Describe data

In [158]:
Toronto.groupby("BOROUGH").describe(include="all").T

BOROUGH       Central Toronto                                              \
                        count unique                                  top   
Latitude                    9    NaN                                  NaN   
Longitude                   9    NaN                                  NaN   
NEIGHBOURHOOD               9      9  Forest Hill North, Forest Hill West   

BOROUGH                                                                    \
              freq    mean        std      min      25%      50%      75%   
Latitude       NaN  43.702  0.0171464  43.6727  43.6896  43.7043  43.7128   
Longitude      NaN -79.399  0.0117277 -79.4169 -79.4057    -79.4 -79.3888   
NEIGHBOURHOOD    1     NaN        NaN      NaN      NaN      NaN      NaN   

BOROUGH         ...    West Toronto  \
                ...          unique   
Latitude        ...             NaN   
Longitude       ...             NaN   
NEIGHBOURHOOD   ...               6   

BOROUGH                                                                    \
                                                        top freq     mean   
Latitude                                                NaN  NaN  43.6527   
Longitude                                               NaN  NaN -79.4493   
NEIGHBOURHOOD  Brockton, Exhibition Place, Parkdale Village    1      NaN   

BOROUGH                                                                
                     std      min      25%      50%      75%      max  
Latitude       0.0112655  43.6368  43.6482  43.6503  43.6591   43.669  
Longitude      0.0240514 -79.4844 -79.4627 -79.4493 -79.4317 -79.4197  
NEIGHBOURHOOD        NaN      NaN      NaN      NaN      NaN      NaN  

[3 rows x 44 columns]

# Draw Graph

In [164]:
from geopy.geocoders import Nominatim 
location = Nominatim().geocode('Toronto, Canada') 

In [ ]:
import folium

map_toronto = folium.Map(location=[location.latitude, location.longitude], zoom_start=9)

# add markers to map
for lat, lng, neighborhood in zip(Toronto['Latitude'], Toronto['Longitude'], Toronto['NEIGHBOURHOOD']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

In [ ]:
CLIENT_ID = 'SZ2220HV3ECG4QJUTHW0NSRLVO0CD54JNAMB3I4OF5RX0MXN' # your Foursquare ID
CLIENT_SECRET = '2WPQ0ZSOWJWOFC1I5HBHMDO2KAQ443QBKUX3BJ1N0S13QSFM' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [ ]:
Toronto_venues = getNearbyVenues( Toronto.Neighbourhood, 
                                   Toronto.Latitude,
                                   Toronto.Longitude )